In [29]:
import csv
import os
from PIL import Image
import piexif

# Set current working directory
os.chdir("g:\\Programming_Playground\\Python_Programming\\GeoMetadataEditor")

# Function to read the metadata from CSV file
def read_metadata(csv_path):
    metadata = {}
    try:
        with open(csv_path, mode='r') as file:
            reader = csv.reader(file)
            for row in reader:
                if row:
                    metadata[row[0]] = row[1]
    except Exception as e:
        print(f"Error reading CSV: {e}")
    return metadata

# Function to encode metadata
def encode_metadata(image_path, output_path, geo_data, metadata, model, date_time, latitude, longitude):
    try:
        print(f"Opening image at path: {image_path}")
        # Open image and initialize EXIF data
        image = Image.open(image_path)

        # Clean any existing EXIF data
        image.info["exif"] = b""

        # Get EXIF data, handling missing or empty EXIF data properly
        exif_data = image.info.get("exif", b"")
        
        if not exif_data:
            print(f"Warning: No EXIF data found in the image '{image_path}'. Generating new EXIF data.")
            exif_data = piexif.dump({})  # Initialize new EXIF data

        exif_dict = piexif.load(exif_data)

        # Set Predefined values (from metadata)
        exif_dict["0th"] = {
            piexif.ImageIFD.Make: metadata.get("Make", "").encode(),
            piexif.ImageIFD.Model: model.encode(),
            piexif.ImageIFD.XResolution: (72, 1),
            piexif.ImageIFD.YResolution: (72, 1),
            piexif.ImageIFD.ResolutionUnit: 2,
            piexif.ImageIFD.Software: metadata.get("Software", "").encode(),
            piexif.ImageIFD.Orientation: 1,
        }

        # Add GPS data from CSV
        gps_ifd = {
            piexif.GPSIFD.GPSLatitudeRef: geo_data["lat_ref"].encode(),
            piexif.GPSIFD.GPSLatitude: convert_to_dms(latitude),
            piexif.GPSIFD.GPSLongitudeRef: geo_data["long_ref"].encode(),
            piexif.GPSIFD.GPSLongitude: convert_to_dms(longitude),
            piexif.GPSIFD.GPSAltitude: (int(geo_data.get("altitude", 0) * 100), 100),
            piexif.GPSIFD.GPSAltitudeRef: 0,
            piexif.GPSIFD.GPSDateStamp: date_time.encode(),
        }
        exif_dict["GPS"] = gps_ifd

        # Additional EXIF data
        exif_dict["Exif"] = {
            piexif.ExifIFD.DateTimeOriginal: date_time.encode(),
            piexif.ExifIFD.DateTimeDigitized: date_time.encode(),
            piexif.ExifIFD.FNumber: (20, 10),
            piexif.ExifIFD.FocalLength: (int(metadata.get("FocalLength", 50)), 100),
            piexif.ExifIFD.ApertureValue: (int(metadata.get("ApertureValue", 2)), 10),
            piexif.ExifIFD.ExposureTime: (1, 105),
            piexif.ExifIFD.ExposureProgram: int(metadata.get("ExposureProgram", 0)),
            piexif.ExifIFD.MeteringMode: int(metadata.get("MeteringMode", 0)),
            piexif.ExifIFD.ISOSpeedRatings: int(metadata.get("ISOSpeedRatings", 400)),
            piexif.ExifIFD.WhiteBalance: int(metadata.get("WhiteBalance", 0)),
            piexif.ExifIFD.SceneCaptureType: int(metadata.get("SceneCaptureType", 0)),
        }

        # Save the new EXIF data
        exif_bytes = piexif.dump(exif_dict)

        # Ensure output directory exists
        output_dir = os.path.dirname(output_path)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Save image with new metadata
        image.save(output_path, "jpeg", exif=exif_bytes)
        print(f"Metadata successfully encoded and saved to {output_path}")

    except Exception as e:
        print(f"Error embedding metadata: {e}")

# Helper function to convert decimal degrees to DMS
def convert_to_dms(value):
    """Convert decimal degrees to degrees, minutes, and seconds for EXIF."""
    degrees = int(value)
    minutes = int((value - degrees) * 60)
    seconds = round((value - degrees - minutes / 60) * 3600, 5)
    return ((degrees, 1), (minutes, 1), (int(seconds * 100), 100))

def main():
    # Define paths based on the working directory
    csv_path = "input/metadata.csv"
    # image_path = "input/input_image.jpeg"
    image_path = "input/input_image2.jpg"
    # output_path = "output/final_image_with_metadata.jpg"
    output_path = "output/final_image_with_metadata2.jpg"

    print(f"CSV Path: {csv_path}")
    print(f"Image Path: {image_path}")
    print(f"Output Path: {output_path}")

    # Read metadata from the CSV file
    metadata = read_metadata(csv_path)

    # Ensure proper keys in metadata (add verification here if necessary)
    required_keys = ["Make", "Software", "model", "FocalLength", "ApertureValue", "ExposureProgram", "MeteringMode", "ISOSpeedRatings", "WhiteBalance", "SceneCaptureType", "lat_ref", "long_ref", "altitude", "latitude", "longitude", "timestamp"]
    missing_keys = [key for key in required_keys if key not in metadata]
    if missing_keys:
        print(f"Missing metadata keys: {missing_keys}")
        return

    # User-specific geo data
    geo_data = {
        "lat_ref": metadata["lat_ref"],
        "long_ref": metadata["long_ref"],
        "altitude": float(metadata["altitude"]),
    }

    # Call the encode function to process and embed metadata in the image
    encode_metadata(image_path, output_path, geo_data, metadata, metadata["model"], metadata["timestamp"], 
                    float(metadata["latitude"]), float(metadata["longitude"]))

if __name__ == "__main__":
    main()


CSV Path: input/metadata.csv
Image Path: input/input_image2.jpg
Output Path: output/final_image_with_metadata2.jpg
Opening image at path: input/input_image2.jpg
Metadata successfully encoded and saved to output/final_image_with_metadata2.jpg


In [30]:
import os
from PIL import Image
import piexif

# Set current working directory
os.chdir("g:\\Programming_Playground\\Python_Programming\\GeoMetadataEditor")

# Function to view EXIF metadata from an image
def view_exif_metadata(image_path):
    try:
        # Open image
        image = Image.open(image_path)
        
        # Get EXIF data
        exif_data = image._getexif()
        
        if exif_data:
            print(f"\nEXIF Metadata for image: {image_path}")
            for tag, value in exif_data.items():
                tag_name = piexif.TAGS.get(tag, tag)
                print(f"{tag_name}: {value}")
        else:
            print("\nNo EXIF data found in the image.")
    except Exception as e:
        print(f"Error opening image: {e}")

# Main function to view metadata from saved image
def main():
    # Define the output image path (updated image with metadata)
    # output_image_path = "output/final_image_with_metadata.jpg"
    output_image_path = "output/final_image_with_metadata2.jpg"
    
    print(f"\nViewing EXIF metadata for the image saved at: {output_image_path}")
    view_exif_metadata(output_image_path)

if __name__ == "__main__":
    main()



Viewing EXIF metadata for the image saved at: output/final_image_with_metadata2.jpg

EXIF Metadata for image: output/final_image_with_metadata2.jpg
34853: {1: 'North', 2: (26.0, 23.0, 9.59), 3: 'East', 4: (83.0, 17.0, 28.71), 5: b'\x00', 6: 31.0, 29: '2025:01:07 13:29:08'}
296: 2
34665: 190
271: Xiaomi
272: Redmi Note 13 5G
305: MediaTek Camera Application
274: 1
282: 72.0
283: 72.0
34850: 0
36867: 2025:01:07 13:29:08
36868: 2025:01:07 13:29:08
37378: 1.0
41987: 0
34855: 100
37383: 5
41990: 0
37386: 3.59
33434: 0.009523809523809525
33437: 2.0


In [10]:
pwd

'g:\\Programming_Playground\\Python_Programming\\GeoMetadataEditor'